<img src="https://docs.qim.dk/qim3d/assets/qim3d-logo.svg" width="256">

# Volumetric data insights using `qim3d`

This notebook provides an initial pipeline for exploring reconstructed volumetric datasets.

We'll begin with image visualization and interactive exploration, and conclude with basic quantitative image analysis.


In [ ]:
import qim3d
import numpy as np

### Select data
Use the variable `DATA_PATH` for your reconstructed dataset

In [ ]:
DATA_PATH = "/dtu/3d-imaging-center/courses/CIL-QIM25_workshop/data/foraminifera/Amphi_13363_10X-40kV-LE1-20s-1p45micro_recon.txm"

### Loading data

In [ ]:
vol = qim3d.io.load(DATA_PATH, progress_bar=True, display_memory_usage=True)

### Initial exploration

In [ ]:
qim3d.viz.slicer(vol, color_bar='slices', slice_axis=1)

In [ ]:
hist = qim3d.viz.histogram(vol, coarseness=10)

### Find data boundaries

The threshold exploration tool can be used to check the threshold levels to be used

In [ ]:
qim3d.viz.threshold(vol, coarseness=4)

In [ ]:
MIN_INTESITY = 30000
MAX_INTENSITY = 60000

In [ ]:
qim3d.viz.slicer(vol, value_min=MIN_INTESITY, value_max=MAX_INTENSITY)

In [ ]:
qim3d.viz.volumetric(vol)

## Make binary volume

In [ ]:
vol_binary = vol > MIN_INTESITY

In [ ]:
qim3d.viz.slicer(vol_binary, image_size=6)

In [ ]:
vol_closed = qim3d.morphology.closing(vol_binary, kernel=3, method='scipy.ndimage')

In [ ]:
qim3d.viz.slicer(vol_closed, image_size=6)

## Extracting the Largest Connected Component

In volumetric image analysis, a connected component refers to a group of neighboring voxels. Typically, if the binary volume well createdx, they are part of the same object. By performing a connected components analysis, we can assign a unique label to each distinct object in the volume.

However, due to background noise or small artifacts, many tiny components may be detected (some as small as a single voxel). These are not relevant in this case.

Fortunately, we often have prior knowledge that the object of interest is the largest structure in the volume. 
Using the `qim3d` library, we can easily isolate and extract this largest component, allowing us to focus on the meaningful part of the data.

In [ ]:
vol_cc = qim3d.segmentation.connected_components(vol_closed)

In [ ]:
color_map = qim3d.viz.colormaps.segmentation(len(vol_cc), style='bright')

In [ ]:
print(f"Total number of components: {len(vol_cc)}")

In [ ]:
qim3d.viz.slicer(vol_cc.labels, slice_axis=0, color_map=color_map, image_size=6)

We can use the volumetric visualization to better understand the components created from noise.

To see the single voxels, make sure to open the `Volume #1` panel and set `samples` to `1000`

In [ ]:
qim3d.viz.volumetric(vol_cc.labels, color_map=color_map, opacity_function="constant")

Here we can filter the *n* largest components, but we're interesred only on the first

In [ ]:
vol_largest_cc = vol_cc.filter_by_largest(1)

In [ ]:
qim3d.viz.slicer(vol_largest_cc, image_size=6)

Now that we're visualizing a solid 3D object, we can improve the view by slicing through the volume. This helps us inspect internal structures more easily.
To do this:

1. Open the Controls panel.
2. Navigate to Clipping Planes and click Add New.
3. Click From Camera [Start] to begin defining the clipping plane based on your current view.
4. Zoom in or rotate the view until the volume is sliced in half or positioned as desired.
5. Once satisfied, click From Camera [Stop] to finalize the clipping plane.

For enhanced rendering:

1. Go to the Volume #1 settings.
2. Increase the number of samples to improve image quality.
3. Enable shadows to add depth and realism to the visualization.

In [ ]:
qim3d.viz.volumetric(vol_largest_cc, opacity_function="constant")

## Local thickness

Now that we've isolated a solid object from the volume, we can begin performing quantitative analysis. One useful metric is Local Thickness.

Local Thickness assigns a value to each voxel based on the diameter of the largest sphere that can fit entirely within the object at that location. This gives us a localized measure of the object's thickness, which can be useful for understanding its structure, identifying thin regions, or comparing different areas.
The qim3d library provides tools to compute this efficiently, allowing us to generate a new volume where each voxel encodes its local thickness.

In [ ]:
# Some clean up
del vol_binary
del vol_closed
del vol_cc

We can perform a subsampling of the dataset to ecrease the processing time

In [ ]:
vol_sub = qim3d.operations.subsample(vol_largest_cc, coarseness=4)
print (f"Subsampled shape: {vol_sub.shape}")

In [ ]:
vol_lt = qim3d.processing.local_thickness(vol_sub, visualize=False)

In [ ]:
qim3d.viz.slicer(vol_lt, color_bar='volume', image_size=6)

In [ ]:
qim3d.viz.volumetric(vol_lt)

In [ ]:
hist_lt = qim3d.viz.histogram(vol_lt, coarseness=10)

In [ ]:
vol_thin = np.where(vol_lt >= 10, vol_lt, 0)

In [ ]:
qim3d.viz.slicer(vol_thin.astype(bool), color_bar='volume', image_size=6)